In [103]:
from langchain_groq import ChatGroq
import re
import httpx

In [104]:
api_key="..."

In [105]:
class Chatbot():
  def __init__(self,system=""):
    self.system=system
    self.message=[]
    if self.system:
      self.message.append({"role":"system","content":system})

  def __call__(self,message):
    self.message.append({"role":"user","content":message})
    result=self.execute()
    self.message.append({"role":"assistant","content":result})
    return result


  def execute(self):
    llm=ChatGroq(model="openai/gpt-oss-20b",api_key=api_key)
    result=llm.invoke(self.message)
    return result.content

In [111]:
action_re=re.compile(r'^Action: (\w+): (.*)')
def wikipedia(query):
  response=httpx.get("https://en.wikipedia.org/w/api.php",
                     params={
                         "action":"query",
                         "list":"search",
                         "srsearch":query,
                         "format":"json"
                     })
  print(response)
  return response.json()["query"]["search"][0]["snippet"]

def calculate(number):
  return eval(number)

In [112]:
known_actions={
    "wikipedia":wikipedia,
    "calculate":calculate
}

In [116]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

**IMPORTANT RULES:**
1. Output MUST be plain text.
2. Do NOT use JSON, XML, or any "tool_call" syntax.
3. Do NOT try to use native function calling. Just write the text.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

In [119]:
def query(question, max_turns=5):
  i=0
  bot=Chatbot(prompt)
  next_prompt=question
  while i<max_turns:
    i+=1
    result=bot(next_prompt)
    print(result)
    actions=[action_re.match(a) for a in result.split('\n') if action_re.match(a)]
    if actions:
      action,action_input=actions[0].groups()
      if action not in known_actions:
        raise Exception(f"Unknown action:{action}:{action_input}")
      print("--running{}{}".format(action,action_input))
      observation=known_actions[action](action_input)
      print("obesrvation:",observation)
      next_prompt=f"Observation:{observation}"
    else:
      return result

In [120]:
print(query("What is Langchain?"))

**LangChain** is an open‑source framework written in Python that helps developers build applications powered by large language models (LLMs).  
Key points:

| Feature | Description |
|---------|-------------|
| **LLM Agnosticism** | Works with many providers (OpenAI, Anthropic, Cohere, etc.). |
| **Modular Chaining** | Lets you string together prompts, tools, and models into reusable “chains.” |
| **Memory & Context** | Provides abstractions for short‑term and long‑term memory, making conversational agents easier to build. |
| **Retrieval‑Augmented Generation** | Integrates retrieval systems (e.g., vector stores, databases) so models can access external knowledge. |
| **Extensible Tooling** | Supports adding custom tools (APIs, databases, web scraping) that LLMs can call. |
| **Community Ecosystem** | Rich set of plugins, examples, and community‑maintained integrations. |

In short, LangChain abstracts common patterns in LLM usage—prompt design, chaining, memory, and external data acce